# 摸鱼神器——Jupyter Lab股票K线查看器

懂的大家都懂，简单讲讲原理吧。数据是通过tushare工具包获取的，图形绘制是pyecharts包实现，后期可添加互动动画效果。这两个包都需要通过pip进行安装。接下来是老朋友pandas和numpy进行数据处理。最后通过Jupyter Lab进行渲染的时候需要设定环境变量。
下面咱们来看看效果吧。

这里使用集成开发环境Anaconda，编译环境是python3.8，另外还需要额外安装三个包，tushare、pyecharts和ta-lib，其中tushare和pyecharts可以通过pip方式进行安装，ta-lib因为是基于Cython的，在Linux环境下可以通过pip进行安装并编译，在Windows环境下如果没安装C语言编译器就只能通过下载编译好的whl文件进行安装。whl文件通过[加州大学尔湾分校](https://www.lfd.uci.edu/~gohlke/pythonlibs)提供的库进行下载，注意这里Ta-Lib有多个版本，既要考虑i386和amd64的区别，也要考虑python版本的区别。

## 一、引入相关包

这里一次性引入tushare, pandas, numpy, pyecharts, ta-lib的相关包，并配置环境变量。

In [1]:
import tushare as ts
import pandas as pd
import numpy as np
from pyecharts import options as opts
from pyecharts.charts import Kline
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts import options as opts
from pyecharts.charts import Line
from pyecharts.charts import Bar, Grid
import datetime as dt
import talib as ta

注意：tushare初次安装之后需要到[Tushare大数据社区](https://tushare.pro/)注册相关账号并获得自己的token，token和用户获取数据的权限相关。具体token的获取和设置方法在网址上有详细介绍，请求数据的方式详见网站“数据接口”功能。
获得自己的token后通过如下方式进行设置，并且实现本机永久有效。

In [2]:
ts.set_token('****你的token信息****')

初始化tushare工具包

In [3]:
pro = ts.pro_api()

## 二、开始接收今日行情信息。。。嘀嘀嘀。。。

这里的股票代码ts_code我只指定了一只股票，ts_code要求是str格式，有同时关注多只股票的请使用英文逗号间隔相关数据，例如'000060.SZ,600000.SH'。神秘的6位代码后面有交易所标志，.SH表示上交所，.SZ表示深交所，以此类推。

start_date和end_date表示获取交易日的开始和终止时间，格式是str，工具包会自动转为date，如果拟采用当期日期自动计算请求数据的起止日期的话，建议转为“yyyymmdd”格式的str。

先确定end_date，使用datetime工具包获取当前系统日期，并转换为str格式。然后通过timedelta方式产生一个日期格式值，将当前日期减去这个值得到start_date。

如果需要分时K线数据，则strftime格式需要进行修改，改为'%Y-%m-%d %H:%M:%S'。freq默认值是D，表示日线，1分钟线使用1min，5分钟线使用5min，15分钟线使用15min，60分钟线使用60min。

adj表示复权类型，默认是不复权None，前复权是qfq，后复权是hfq。

ma表示请求的移动加权数据类型，不输入ma则不返回移动加权数据，输入ma数据后返回移动加权的价格和交易量。

In [4]:
stock1='000060.SZ'

In [5]:
end_date=dt.datetime.now().date().strftime('%Y%m%d')

In [6]:
start_date=(dt.datetime.now().date()+dt.timedelta(days=-360)).strftime('%Y%m%d')

取数接口要一次性把价格K线、移动加权平均（MA）、成交量（VOL）取到，就使用pro_bar接口。

In [7]:
df=ts.pro_bar(ts_code=stock1,start_date=start_date,end_date=end_date,asset='E',freq='D',adj='qfq',ma=[5,10,20,30])

df=df.sort_index(ascending=False)

df['macd'], df['macdsignal'], df['macdhist'] = ta.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)

df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,...,ma_v_5,ma10,ma_v_10,ma20,ma_v_20,ma30,ma_v_30,macd,macdsignal,macdhist
236,000060.SZ,20191216,4.0497,4.0791,4.0203,4.0693,4.0301,0.0392,0.9727,242068.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,000060.SZ,20191217,4.0889,4.1380,4.0791,4.1183,4.0693,0.0490,1.2041,286527.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,000060.SZ,20191218,4.1282,4.1380,4.0889,4.0987,4.1183,-0.0196,-0.4759,167023.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,000060.SZ,20191219,4.0889,4.1085,4.0791,4.1085,4.0987,0.0098,0.2391,137798.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,000060.SZ,20191220,4.0987,4.1282,4.0497,4.0693,4.1085,-0.0392,-0.9541,157441.35,...,198171.874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,000060.SZ,20201201,4.9400,5.0300,4.8600,4.9900,4.9800,0.0100,0.2008,997780.76,...,1198411.162,4.983,1480283.493,4.7030,1.029659e+06,4.5307,749476.0567,0.185292,0.167881,0.017411
3,000060.SZ,20201202,5.0000,5.1200,4.9100,4.9500,4.9900,-0.0400,-0.8016,1063333.52,...,1118413.230,4.993,1466533.765,4.7405,1.072474e+06,4.5537,780017.7500,0.176960,0.169697,0.007263
2,000060.SZ,20201203,4.8900,4.9000,4.7500,4.7800,4.9500,-0.1700,-3.4343,911764.71,...,1095066.320,4.996,1474874.437,4.7670,1.107143e+06,4.5723,806087.5830,0.154854,0.166728,-0.011874
1,000060.SZ,20201204,4.7500,4.8200,4.7200,4.8000,4.7800,0.0200,0.4184,536832.05,...,1052915.238,4.984,1425799.091,4.7925,1.118293e+06,4.5917,819623.0813,0.137365,0.160855,-0.023491


## 三、绘图能手上场

下面开始画图了，先画K线图。pyecharts不接收dataframe格式数据，只接收list数据，所以dataframe中的columns和row以及index都没有意义，如有index要转入建议通过reset_index方式重置索引。

pyecharts编程和画图的逻辑相对比较符合我国人民的思想，先声明要画什么样的图形，条形图就是Bar()、折线图就是line()，K线图就是Kline()。。。

先进行图形类型声明，然后进行图像属性设定。

.add_xaxis表示设定X轴，因为pyecharts对于X轴数据没有指定标题，所以直接导入list数据就行。

.add_yaxis表示设定Y轴，pyecharts要求每一条Y轴数据都要有一个名字，因为K线图中这次只演示一只股票('000060.SZ')，所以只添加一条K线就行。如果要在一张图里对比多只股票，就多添加几行.add_yaxis。

设定格式是.add_yaxis('名字',数据)，数据传入list格式。

.set_global_opts是对于pyecharts图形的基础属性设置，splitline_opts表示分割线属性。title_opts设定图形的标题属性。tooltip_opts设定图形上跟踪光标的指示器，其中trigger='axis'表示跟踪坐标轴触发，axis_pointer_type='line'表示指示器是线形。datazoom_opts表示图形缩放属性，is_show表示是否启用缩放，typy_有两种方式'slider'和'inside'两种，'slider'在图形下方展示缩放滚动条，'inside'通过滚轮进行图形缩放。


In [8]:
kline = (
    Kline()
    .add_xaxis(xaxis_data=df['trade_date'].values.tolist())
    .add_yaxis(series_name='kline',y_axis=df[['open','close','high','low']].values.tolist())
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitline_opts=opts.SplitLineOpts(is_show=True)
        ),
        xaxis_opts=opts.AxisOpts(
            is_scale=True,
            type_='category',
            axisline_opts=opts.AxisLineOpts(is_on_zero=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
        ),
        title_opts=opts.TitleOpts(title=stock1),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="line"),
        datazoom_opts=[opts.DataZoomOpts(
            is_show=True,
            type_='inside',
            )
        ]
    )
)


这里要注意，K线图传入是Y轴数据是一次性传入4个，开盘价open、收盘价close、最高价high、最低价low。pyecharts因为只接受list数据，所以不区分X轴和Y轴数据的匹配关系，因此传入前一定要严格检查X轴和Y轴数据的匹配关系，不要出现顺序错乱。

然后咱们来看看K线的移动加权平均线怎么画。咱们这次取到了ma5、ma10、ma20、ma30四组移动加权平均值，先画出来价格的移动加权平均线，然后再想办法叠加到K线图上。

In [9]:
kline_line = (
    Line()
    .add_xaxis(xaxis_data=df['trade_date'].values.tolist())
    .add_yaxis(series_name='MA5',
               y_axis=df['ma5'],
              is_smooth=True,
               label_opts=opts.LabelOpts(
                       is_show=False)
               )
    .add_yaxis(series_name='MA10',
               y_axis=df['ma10'],
              is_smooth=True,
               label_opts=opts.LabelOpts(
                       is_show=False)
               )
    .add_yaxis(series_name='MA20',
               y_axis=df['ma20'],
              is_smooth=True,
               label_opts=opts.LabelOpts(
                       is_show=False)
               )
    .add_yaxis(series_name='MA30',
               y_axis=df['ma30'],
              is_smooth=True,
               label_opts=opts.LabelOpts(
                       is_show=False)
               )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="category",
            grid_index=1,
            ),
        yaxis_opts=opts.AxisOpts(
            grid_index=1,
            axisline_opts=opts.AxisLineOpts(is_on_zero=False),
            axistick_opts=opts.AxisTickOpts(is_show=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
            axislabel_opts=opts.LabelOpts(is_show=True),            
            )
    )
)

接下来把移动加权平均线叠加到K线图上

In [10]:
overlap_kline_line = kline.overlap(kline_line)

接下来开始画成交量图，成交量图是条形图。同样X轴和Y轴传入相关数据，并设置好标题即可。

In [11]:
bar_1 = (
    Bar()
    .add_xaxis(df['trade_date'].values.tolist())
    .add_yaxis(series_name='volume',
               y_axis=df['vol'].values.tolist(),
               xaxis_index=None,
               yaxis_index=None,
               label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
                xaxis_opts=opts.AxisOpts(
                type_="category",
                axislabel_opts=opts.LabelOpts(is_show=False),
#               datazoom_opts=[opts.DataZoomOpts(is_show=True,type_='slider')]
                )
    )
)


接下来画MACD图，hist数据画条形图，DIF和DEA数据画折线图，再进行叠加。

In [12]:
bar_2 = (
        Bar()
        .add_xaxis(df['trade_date'].values.tolist())
        .add_yaxis(series_name='macdhist',
                   y_axis=df['macdhist'].values.tolist(),
                   label_opts=opts.LabelOpts(
                       is_show=False))
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(is_show=True)
            ),
            yaxis_opts=opts.AxisOpts(
                axisline_opts=opts.AxisLineOpts(is_on_zero=True),
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
                axislabel_opts=opts.LabelOpts(is_show=True),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
            
        )
        
)

line_2 = (
            Line()
            .add_xaxis(df['trade_date'].values.tolist())
            .add_yaxis(series_name='DIF',
            y_axis=df['macd'].values.tolist(),
            label_opts=opts.LabelOpts(
                       is_show=False))
            .add_yaxis(series_name='DEA',
            y_axis=df['macdsignal'].values.tolist(),
            label_opts=opts.LabelOpts(
                       is_show=False))
            .set_global_opts(
                xaxis_opts=opts.AxisOpts(
                    axislabel_opts=opts.LabelOpts(is_show=True),
                ),
                yaxis_opts=opts.AxisOpts(
                    axislabel_opts=opts.LabelOpts(is_show=True),)
            )
)

接下来把bar_2和line_2合并成一张图

In [13]:
overlap_bar_line = bar_2.overlap(line_2)

把上面的overlap_kline_line、bar_1、overlap_bar_line三合一。通过Grid图形组合工具来实现。Grid在pyecharts处理时实际作为了一种图形对象来处理，同样先声明图形类型Grid()，然后开始添加要进行组合的图。

In [14]:
grid = (
        Grid()
        .add(overlap_kline_line,
             grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%", height="60%"),
            )
        .add(bar_1,
            grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%", pos_top="71%", height="10%"),
            )
        .add(overlap_bar_line,
            grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%", pos_top="82%", height="14%"),
            )
)

最后对组合图grid加载javascript方法，并使用render_notebook()进行显示。由于jupyter lab的限制，这两部不能在同一个cell中执行，因此分开两个cell填写。

In [15]:
grid.load_javascript()

In [16]:
grid.render_notebook()